In [6]:
import pandas as pd
import numpy as np
import spacy
from tqdm import tqdm
import re
import time
import pickle
import en_core_web_sm
pd.set_option('display.max_colwidth', 200)
from nltk.corpus import stopwords

In [2]:
df = pd.read_csv("amazon_electronics.csv")

In [3]:
print (df.shape)

(5003, 9)


In [4]:
df.head(3)

,askerID,questionText,answererID,helpful,answerText,8,9,sent_score,sent_score_quant
0,AMD370KY9I1WK,"Good for rabbit ears? I live in a basement apartment and use rabbit ears to get my TV reception, which is poor at best. I was wondering if this would help me get the weaker channels in better or ...",A1GNC9LPUQ8HTG,"[6, 6]","I purchased this Motorola signal booster for my ""rabbit ears""-connected LCD tv. Received the Motorola on 1/31 and noticed it did NOT improve the tv's weak signal. When I called Motorola's tech su...","['purchased', 'motorola', 'signal', 'booster', 'rabbit', 'ear', 'connected', 'lcd', 'tv', 'received', 'motorola', 'noticed', 'improve', 'tv', 'weak', 'signal', 'called', 'motorola', 'tech', 'suppo...",purchased motorola signal booster rabbit ear connected lcd tv received motorola noticed improve tv weak signal called motorola tech support evening told particular model work cable satellite hook ...,0.032676,4
1,AMD370KY9I1WK,"Good for rabbit ears? I live in a basement apartment and use rabbit ears to get my TV reception, which is poor at best. I was wondering if this would help me get the weaker channels in better or ...",A3NYJZ71CESSP8,"[3, 3]",When an amplifier receives a bad signal all you get is an amplified bad signal. It's main purpose is to boost a (good) signal that has been weakened by long cable runs or too many splitter.,"['amplifier', 'receives', 'bad', 'signal', 'get', 'amplified', 'bad', 'signal', 'main', 'purpose', 'boost', 'good', 'signal', 'ha', 'weakened', 'long', 'cable', 'run', 'many', 'splitter']",amplifier receives bad signal get amplified bad signal main purpose boost good signal ha weakened long cable run many splitter,0.003088,2
2,AMD370KY9I1WK,"Good for rabbit ears? I live in a basement apartment and use rabbit ears to get my TV reception, which is poor at best. I was wondering if this would help me get the weaker channels in better or ...",A3UD50M7M72150,"[1, 1]","hey, i'm in the same prob here, looking for something to boost my weak signal for my regular tv. I don't use cable, just rabbit ears. Any sugestion which antenna is best or should i go look for bo...","['hey', 'prob', 'looking', 'something', 'boost', 'weak', 'signal', 'regular', 'tv', 'use', 'cable', 'rabbit', 'ear', 'sugestion', 'antenna', 'best', 'go', 'look', 'booster']",hey prob looking something boost weak signal regular tv use cable rabbit ear sugestion antenna best go look booster,0.039464,4


In [5]:
df = df.dropna()
df.sent_score_quant = df.sent_score_quant.astype(int)
df.answerText = df.answerText.str.lower()

In [33]:
punctuation = '!"#$%&()*+-/:;<=>?@[\\]^_`{|}~'
stop_words = set(stopwords.words('english'))

df['clean_text'] = df.answerText.apply(lambda x: ''.join(ch for ch in x if ch not in set(punctuation)))

# remove numbers
df['clean_text'] = df['clean_text'].str.replace("[0-9]", " ")

# remove whitespaces
df['clean_text'] = df['clean_text'].apply(lambda x:' '.join(x.split()))

df['clean_text'] = df.clean_text.apply(lambda x: ' '.join([i for i in x.split() if i not in stop_words]))

In [94]:
max_len = 100
batch_size = 2

In [16]:
#df['clean_text'] = df['clean_text'].apply(lambda x: x.split())

In [10]:
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# function to lemmatize text
def lemmatization(texts):
    output = []
    for i in texts:
        s = [token.lemma_ for token in nlp(i)]
        output.append(' '.join(s))
    return output

df['clean_text'] = lemmatization(df['clean_text'])

In [64]:
new_X = []
for i in range(df.shape[0]):
    seq = df.clean_text.iloc[i].split()
    new_seq = []
    for j in range(max_len):
        try:
            new_seq.append(seq[j])
        except:
            new_seq.append("__PAD__")
    new_X.append(new_seq)
df_text = np.array(new_X)
print (df_text.shape)

(5003, 100)


In [78]:
import tensorflow as tf
import tensorflow_hub as hub
from keras import backend as K
import keras.layers as layers
from keras.models import Model, load_model
from keras.engine import Layer
from keras import initializers, regularizers, constraints

In [13]:
# Initialize session
sess = tf.Session()
K.set_session(sess)

In [37]:
class ElmoEmbeddingLayer(Layer):
    def __init__(self, **kwargs):
        self.dimensions = 1024
        self.trainable=True
        super(ElmoEmbeddingLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.elmo = hub.Module('https://tfhub.dev/google/elmo/2', trainable=self.trainable,
                               name="{}_module".format(self.name))

        self.trainable_weights += K.tf.trainable_variables(scope="^{}_module/.*".format(self.name))
        super(ElmoEmbeddingLayer, self).build(input_shape)

    def call(self, x, mask=None):
        result = self.elmo(K.squeeze(K.cast(x, tf.string), axis=1),
                      as_dict=True,
                      signature='default',
                      )['default'] #K.squeeze(K.cast(x, tf.string), axis=1)
        return result

    def compute_mask(self, inputs, mask=None):
        return K.not_equal(inputs, '--PAD--')

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.dimensions)

In [49]:
def ELMoEmbedding(x):
    elmo_model = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)
    return elmo_model(tf.squeeze(tf.cast(x, tf.string)), signature="default", as_dict=True)["default"]

In [96]:
def ELMoEmbedding(x):
    elmo_model = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)
    return elmo_model(inputs={
                            "tokens": tf.squeeze(tf.cast(x, tf.string)),
                            "sequence_len": tf.constant(batch_size*[max_len])
                      },
                      signature="tokens",
                      as_dict=True)["elmo"]

In [76]:
class Attention(Layer):
     # Input shape 3D tensor with shape: `(samples, steps, features)`.
     # Output shape 2D tensor with shape: `(samples, features)`.

    def __init__(self, step_dim,W_regulizer = None,b_regulizer = None,
                 W_constraint = None, b_constraint = None,bias = True,**kwargs):
        
        self.W_regulizer = W_regulizer
        self.b_regulizer = b_regulizer
        
        self.W_constraint = W_constraint
        self.b_constraint = b_constraint
        
        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0
        self.init = initializers.get('glorot_uniform')
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3
        # Create a trainable weight variable for this layer.
        self.W = self.add_weight(name='kernel', 
                                      shape=(input_shape[-1],),
                                      initializer= self.init,
                                      constraint = self.W_constraint,
                                      regularizer = self.W_regulizer)
        
        self.features_dim = input_shape[-1]
        
        if self.bias:
            self.b = self.add_weight((input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regulizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None
        super(Attention, self).build(input_shape)  

    
    def call(self, x, mask=None):
      
        features_dim = self.features_dim
        step_dim = self.step_dim

        eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)), K.reshape(self.W, (features_dim, 1))), (-1, step_dim))

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        # apply mask after the exp. will be re-normalized next
        if mask is not None:
           
            a *= K.cast(mask, K.floatx())

        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
    
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0],  self.features_dim

In [50]:
# Create datasets (Only take up to 150 words for memory)
#df['clean_text2'] = df['clean_text'].apply(lambda x: x.split())
#df_text = df['clean_text'].tolist()
#df_text = np.array(df_text, dtype=object)[:, np.newaxis]
#print (df_text.shape)
#df_label = df['sent_score_quant'].tolist()

(5003, 1)


In [54]:
import keras

In [66]:
df_label = keras.utils.to_categorical(df.sent_score_quant.values)
df_label.shape

(5003, 6)

In [83]:
#!git clone https://github.com/thushv89/attention_keras.git

Cloning into 'attention_keras'...
remote: Enumerating objects: 99, done.
remote: Total 99 (delta 0), reused 0 (delta 0), pack-reused 99
Unpacking objects: 100% (99/99), done.


In [87]:
from keras_self_attention import SeqSelfAttention

In [97]:
input_text = layers.Input(shape=(max_len,), dtype=tf.string)
embedding = layers.Lambda(ELMoEmbedding, output_shape=(max_len, 1024))(input_text)

dense = layers.TimeDistributed(layers.Dense(128, activation='relu'))(embedding)
#attn_layer = AttentionLayer(name='attention_layer')
#attn_out, attn_states = attn_layer([dense, dense])

attention = SeqSelfAttention(attention_activation='relu')(dense) #Attention(max_len)(dense)
dense = layers.Flatten()(attention)
pred = layers.Dense(df_label.shape[1], activation='softmax')(dense)

model = Model(inputs=[input_text], outputs=pred)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

embedding_model = Model(inputs=[input_text], outputs=embedding)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0912 22:24:16.018298 4508337600 saver.py:1483] Saver not created because there are no variables in the graph to restore


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_32 (InputLayer)        (None, 100)               0         
_________________________________________________________________
lambda_27 (Lambda)           (None, 100, 1024)         0         
_________________________________________________________________
time_distributed_13 (TimeDis (None, 100, 128)          131200    
_________________________________________________________________
seq_self_attention_4 (SeqSel (None, 100, 128)          8257      
_________________________________________________________________
flatten_5 (Flatten)          (None, 12800)             0         
_________________________________________________________________
dense_46 (Dense)             (None, 6)                 76806     
Total params: 216,263
Trainable params: 216,263
Non-trainable params: 0
_________________________________________________________________


In [98]:
model.fit(df_text[:100], df_label[:100],epochs=1,batch_size=batch_size,validation_split=.2)

Train on 80 samples, validate on 20 samples
Epoch 1/1
80/80 [==============================] - 284s 4s/step - loss: 11.9475 - acc: 0.2250 - val_loss: 12.8945 - val_acc: 0.2000


In [99]:
embedding_model = Model(inputs=[input_text], outputs=embedding)

In [101]:
embedding_df = embedding_model.predict(df_text[:100],batch_size=batch_size)

In [102]:
embedding_df.shape

(100, 100, 1024)

In [104]:
pred = model.predict(df_text[:100], batch_size=batch_size)

In [105]:
pred.argmax(axis=1)

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [107]:
df_label[:100]

array([[0., 0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 1.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 1.],
       [0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0., 0.],
       [0.

In [31]:
model.save('elmo_ppm.h5')

In [38]:
embedding_df[0].shape

(1024,)

In [108]:
embedding_df[0]

array([[-0.06964785,  0.49956885,  0.5536779 , ...,  0.20250148,
        -0.14449076,  0.29933494],
       [-0.1393673 ,  0.3974867 , -0.11697337, ...,  0.11262876,
        -0.31328064, -0.12441117],
       [ 0.3715643 ,  0.4552157 , -1.0826524 , ..., -0.18569992,
         0.07689745, -0.38033748],
       ...,
       [-0.84628683,  0.15489116, -0.5080668 , ..., -0.5052352 ,
        -0.21507479, -0.6429738 ],
       [-0.8443383 ,  0.149794  , -0.50770366, ..., -0.29923922,
        -0.32143813, -0.7706367 ],
       [-0.84252095,  0.14497095, -0.5073981 , ..., -0.1159803 ,
        -0.17632908, -0.42306253]], dtype=float32)

In [109]:
df_text[0]

array(['purchased', 'motorola', 'signal', 'booster', 'rabbit',
       'earsconnected', 'lcd', 'tv.', 'received', 'motorola', 'noticed',
       'improve', "tv's", 'weak', 'signal.', 'called', "motorola's",
       'tech', 'support', 'evening,', 'told', 'particular', 'model',
       'works', 'cable', 'satellite', 'hookups.', 'save', 'money.', 'buy',
       'stronger', 'antenna.chris', '__PAD__', '__PAD__', '__PAD__',
       '__PAD__', '__PAD__', '__PAD__', '__PAD__', '__PAD__', '__PAD__',
       '__PAD__', '__PAD__', '__PAD__', '__PAD__', '__PAD__', '__PAD__',
       '__PAD__', '__PAD__', '__PAD__', '__PAD__', '__PAD__', '__PAD__',
       '__PAD__', '__PAD__', '__PAD__', '__PAD__', '__PAD__', '__PAD__',
       '__PAD__', '__PAD__', '__PAD__', '__PAD__', '__PAD__', '__PAD__',
       '__PAD__', '__PAD__', '__PAD__', '__PAD__', '__PAD__', '__PAD__',
       '__PAD__', '__PAD__', '__PAD__', '__PAD__', '__PAD__', '__PAD__',
       '__PAD__', '__PAD__', '__PAD__', '__PAD__', '__PAD__', '__PAD__'

In [110]:
from sklearn.metrics.pairwise import cosine_distances

In [117]:
df_text[0][cosine_distances(embedding_df[0])[1].argsort()]

array(['motorola', 'motorola', 'signal', 'earsconnected', "motorola's",
       'booster', 'lcd', 'rabbit', 'tech', '__PAD__', 'cable',
       'satellite', 'purchased', '__PAD__', 'model', '__PAD__', '__PAD__',
       'antenna.chris', '__PAD__', 'evening,', 'support', '__PAD__',
       '__PAD__', '__PAD__', '__PAD__', "tv's", 'received', '__PAD__',
       '__PAD__', '__PAD__', '__PAD__', '__PAD__', '__PAD__', '__PAD__',
       '__PAD__', '__PAD__', '__PAD__', '__PAD__', '__PAD__', '__PAD__',
       '__PAD__', '__PAD__', '__PAD__', '__PAD__', '__PAD__', '__PAD__',
       '__PAD__', '__PAD__', 'tv.', '__PAD__', '__PAD__', '__PAD__',
       '__PAD__', '__PAD__', '__PAD__', '__PAD__', '__PAD__', '__PAD__',
       '__PAD__', '__PAD__', '__PAD__', '__PAD__', '__PAD__', '__PAD__',
       '__PAD__', '__PAD__', '__PAD__', '__PAD__', '__PAD__', '__PAD__',
       '__PAD__', '__PAD__', '__PAD__', '__PAD__', '__PAD__', '__PAD__',
       '__PAD__', '__PAD__', '__PAD__', 'stronger', '__PAD__', '__PAD_

In [118]:
df_text[1]

array(['amplifier', 'receives', 'bad', 'signal', 'get', 'amplified',
       'bad', 'signal.', 'main', 'purpose', 'boost', 'good', 'signal',
       'weakened', 'long', 'cable', 'runs', 'many', 'splitter.',
       '__PAD__', '__PAD__', '__PAD__', '__PAD__', '__PAD__', '__PAD__',
       '__PAD__', '__PAD__', '__PAD__', '__PAD__', '__PAD__', '__PAD__',
       '__PAD__', '__PAD__', '__PAD__', '__PAD__', '__PAD__', '__PAD__',
       '__PAD__', '__PAD__', '__PAD__', '__PAD__', '__PAD__', '__PAD__',
       '__PAD__', '__PAD__', '__PAD__', '__PAD__', '__PAD__', '__PAD__',
       '__PAD__', '__PAD__', '__PAD__', '__PAD__', '__PAD__', '__PAD__',
       '__PAD__', '__PAD__', '__PAD__', '__PAD__', '__PAD__', '__PAD__',
       '__PAD__', '__PAD__', '__PAD__', '__PAD__', '__PAD__', '__PAD__',
       '__PAD__', '__PAD__', '__PAD__', '__PAD__', '__PAD__', '__PAD__',
       '__PAD__', '__PAD__', '__PAD__', '__PAD__', '__PAD__', '__PAD__',
       '__PAD__', '__PAD__', '__PAD__', '__PAD__', '__PAD__', '__

In [120]:
df_text[1][cosine_distances(embedding_df[0])[0].argsort()]

array(['amplifier', 'main', '__PAD__', 'receives', '__PAD__', 'boost',
       'amplified', 'cable', 'signal', '__PAD__', 'bad', 'purpose', 'bad',
       '__PAD__', 'signal.', 'good', 'runs', 'get', 'splitter.',
       '__PAD__', '__PAD__', '__PAD__', 'many', '__PAD__', 'signal',
       '__PAD__', '__PAD__', '__PAD__', '__PAD__', 'weakened', 'long',
       '__PAD__', '__PAD__', '__PAD__', '__PAD__', '__PAD__', '__PAD__',
       '__PAD__', '__PAD__', '__PAD__', '__PAD__', '__PAD__', '__PAD__',
       '__PAD__', '__PAD__', '__PAD__', '__PAD__', '__PAD__', '__PAD__',
       '__PAD__', '__PAD__', '__PAD__', '__PAD__', '__PAD__', '__PAD__',
       '__PAD__', '__PAD__', '__PAD__', '__PAD__', '__PAD__', '__PAD__',
       '__PAD__', '__PAD__', '__PAD__', '__PAD__', '__PAD__', '__PAD__',
       '__PAD__', '__PAD__', '__PAD__', '__PAD__', '__PAD__', '__PAD__',
       '__PAD__', '__PAD__', '__PAD__', '__PAD__', '__PAD__', '__PAD__',
       '__PAD__', '__PAD__', '__PAD__', '__PAD__', '__PAD__', '__